# Proof of Concept of a generalizable titration algorithm

# Concepts
## Rule
A rule has a `parameter`, `operation`, and a `threshold` such that running the `operation` on `parameter` and `threshold` always returns a boolean value (True or False). This value can be computed by running the method `is_satisfied`.

## Action
A specific recommendation or action. It has three methods that can be defined for each specific action type: the first returns a text version, the second a clickable button, and the third implements what should be done when the button is clicked.

## Rule With Actions
Extends `Rule` but also suggests a list of `Action`s depending on whether it is satisfied or not. These are defined by two attributes: `actions_when_satisfied` and `actions_when_not_satisfied`. In addition to `Action`s, these two attributes can hold other Rules with Actions. The method `get_suggested_actions` evaluates the `Rule` and selects the appropriate list of actions. If the list contains other Rules With Actions, these are evaluated recursively until the list only contains `Action`s.

## Dosing Ladder

## Titrator
The actual engine running the titration process. Has `checkpoints`, which is a list of `Rule(s)WithActions`. `checkpoints` indicates both initiation and titration rules, but these can be specified separately by defining `inititation_checkpoints` and `titration_checkpoints`. 

In [ ]:
#|default_exp titrations

In [ ]:
#|export
from typing import List
from titrations.basics import *

In [2]:
#|export

class DosingLadder:
    def __init__(self, steps : List[Medication]):
        self.steps = steps
        self.ingredient = self.steps[0].ingredient

    def _current_dose_index(self, current_med):
        index = next((i for i, med in enumerate(self.steps) if med.dose == current_med.dose), None)
        return index
    
    def _at_highest_dose(self, current_med):
        return current_med.dose == self.highest_dose.dose
    
    def next_dose_up(self, current_med):
        current_dose_index = self._current_dose_index(current_med)
        return self.steps[current_dose_index + 1]

    def next_dose_down(self, current_med):
        current_dose_index = self._current_dose_index(current_med)
        return self.steps[current_dose_index - 1]

    @property
    def lowest_dose(self):
        return self.steps[0]

    @property
    def highest_dose(self):
        return self.steps[-1]

In [44]:
#|export
class DosingLadderGroup:
    def __init__(self, dosing_ladders : List[DosingLadder]):
        lengths = map(lambda ladder: len(ladder.steps), dosing_ladders)
        assert len(set(lengths)) == 1, "Dosing ladders of unequal length."
        self.ladders = {ladder.ingredient.name : ladder for ladder in dosing_ladders}

    @property
    def lowest_doses(self):
        return {ingredient_name : self.ladders[ingredient_name].steps[0] for ingredient_name in self.ladders}
    
    @property
    def highest_doses(self):
        return {ingredient_name : self.ladders[ingredient_name].steps[-1] for ingredient_name in self.ladders}

In [4]:
#|export
class Action:
    """
    This is a base class for 'Action' classes to build upon.
    """
    def __init__(self, patient : Patient, dosing_ladder : DosingLadder):
        self.patient = patient
        self.dosing_ladder = dosing_ladder
        self.ingredient = dosing_ladder.ingredient

    def suggest(self):
        pass

    def perform(self):
        pass

class Start(Action):
    """
    Start a new medication.
    """
    def suggest(self):
        return f"Start {self.dosing_ladder.lowest_dose}."

    def perform(self):
        pass

class DoNotStart(Action):
    """
    Do not start a new medication.
    """
    def suggest(self):
        return f"Do not start {self.ingredient.name}."

    def perform(self):
        pass

class StepUp(Action):
    """
    Step up one dose on the dosing ladder.
    """
    def suggest(self):
        current_dose = self.patient.current_dose_of(self.ingredient)
        next_dose = self.dosing_ladder.next_dose_up(current_dose)
        return f"Increase {self.ingredient.name} to {next_dose}."

    def perform(self):
        pass

class StepDown(Action):
    """
    Step down one dose on the dosing ladder.
    """
    def suggest(self):
        current_dose = self.patient.current_dose_of(self.ingredient)
        next_dose = self.dosing_ladder.next_dose_down(current_dose)
        return f"Decrease {self.ingredient.name} to {next_dose}."

    def perform(self):
        pass

class Continue(Action):
    """
    Continue the medication at the same dose.
    """
    def suggest(self):
        current_dose = self.patient.current_dose_of(self.ingredient)
        return f"Continue {current_dose}."

    def perform(self):
        pass

class Stop(Action):
    """
    Stop the medication.
    """
    def suggest(self):
        return f"Stop {self.ingredient.name}."

    def perform(self):
        pass

class MarkMaxDose(Action):
    """
    Mark current dose as maxiumum tolerated dose.
    """
    def suggest(self):
        current_dose = self.patient.current_dose_of(self.ingredient)
        return f"Mark {current_dose} as maximum tolerated dose."
    
    def perform(self):
        pass

class ReportReaction(Action):
    """
    File an adverse reaction record.
    """
    def suggest(self):
        return f"File an adverse reaction to {self.ingredient.name}"

    def perform(self, description="reaction"):
        # In real life, this could be opening a modal for entering adverse reactions instead
        self.patient.reactions.append(
            Reaction(self.ingredient, description),
        )

In [5]:
#|export
import operator

class Rule:
    def satisfies_rule(self, patient: Patient) -> bool:
        pass

class Intolerance:
    operators = {
        "gt": operator.gt,
        "gte": operator.ge,
        "lt": operator.lt,
        "lte": operator.le,
        "eq": operator.eq,
        "neq": operator.ne,
        "in": operator.contains,
    }
    success_actions : List[Action] = []
    failure_actions : List[Action] = []

    def __init__(self,
                 parameter:str, operation:str, threshold,
                 additional_success_actions: List[Action] = [],
                 additional_failure_actions: List[Action] = []):
        assert parameter in VALID_PARAMETERS, "Not a valid parameter"
        assert operation in self.operators, f"Invalid operator {operation}"
        self.parameter = parameter
        self.operation = operation
        self.threshold = threshold
        self.success_actions.extend(additional_success_actions)
        self.failure_actions.extend(additional_failure_actions)

    def meets_threshold(self, patient):
        patient_value = getattr(patient, self.parameter, None)
        if not patient_value: return False
        return self.operators[self.operation](patient_value, self.threshold) if self.operation != "in" else self.operators[self.operation](self.threshold, patient_value)
    
    def suggested_actions(self, patient) -> List[Action]:
        return self.success_actions if self.meets_threshold(patient) else self.failure_actions
    
    def __repr__(self) -> str:
        return f"{self.parameter} {self.operation} {self.threshold}"

In [6]:
#|export
class ConditionalIntolerance(Intolerance):
    def __init__(self,
                 parameter: str, operation, threshold,
                 condition_parameter: str,
                 condition_operation,
                 condition_threshold):
        super().__init__(parameter, operation, threshold)

        assert condition_parameter in VALID_PARAMETERS
        self.condition_parameter = condition_parameter
        self.condition_operation = condition_operation
        self.condition_threshold = condition_threshold

    def condition_is_met(self, patient):
        patient_value = getattr(patient, self.condition_parameter)
        return self.operators[self.condition_operation](patient_value, self.condition_threshold)
    
    def meets_threshold(self, patient):
        return super().meets_threshold(patient) if self.condition_is_met(patient) else False

In [7]:
#|export
class ClassLimitingIntolerance(Intolerance):
    success_actions = [Stop, ReportReaction]

class TitrationLimitingIntolerance(Intolerance):
    success_actions = [Continue, StepDown, MarkMaxDose]

class NonTitrationLimitingIntolerance(Intolerance):
    pass

In [8]:
#|export
class Titrator:
    def __init__(self, patient : Patient, dosing_ladder : DosingLadder = None):
        self.patient = patient
        
        if dosing_ladder:
            self.dosing_ladder = dosing_ladder

        if getattr(self, 'dosing_ladder', None):
            self.ingredient = self.dosing_ladder.ingredient
    
    medication_target_options = (
        ('MAX', 'Maximum tolerated'),
        ('PARAMETER', 'Titrate to a parameter'),
    )
    titration_target: Rule

    dosing_ladder: DosingLadder
    intolerances: List[Intolerance] = []

    def _satisfied_rules(self):
        return list(filter(lambda rule: rule.meets_threshold(self.patient), self.intolerances))

    def can_titrate(self):
        outputs = map(lambda rule: not rule.meets_threshold(self.patient), self.intolerances)
        return all(outputs)
    
    def get_suggestion_texts(self):
        suggested_actions = self.get_suggested_actions()
        return [action.suggest() for action in suggested_actions]

    def get_suggested_actions(self):
        # TODO: These should be derived from the `intolerance` rules rather than specified manually.
        if self.can_titrate():
            return [StepUp(self.patient, self.dosing_ladder)] if self.patient.on_medication(self.ingredient) else \
                  [Start(self.patient, self.dosing_ladder)]
        else:
            from itertools import chain
            action_lists = [rule.success_actions for rule in self._satisfied_rules()]
            return [action(self.patient, self.dosing_ladder) for action in set(chain.from_iterable(action_lists))]

            ## return [Continue(self.patient, self.dosing_ladder) if self.patient.on_medication(self.ingredient) else \
            ##        [DoNotStart(self.patient, self.dosing_ladder)]]

In [55]:
#|export
import itertools

class TitratorGroup:
    patient: Patient
    titrators: List[Titrator]
    exclusive: bool # being on a medication in any of the member ladders precludes going up any other ladder

    def __init__(self, patient : Patient, titrators : List[Titrator], exclusive : bool = False):
        self.patient = patient
        self.titrators = titrators
        self.exclusive = exclusive

    @classmethod
    def from_titrator_types(cls, patient : Patient, titrator_classes : List[type[Titrator]], exclusive : bool = False):
        titrators = [tc(patient, dosing_ladder=tc.dosing_ladder) for tc in titrator_classes]
        return cls(patient, titrators, exclusive)
    
    def _satisfied_rules(self):
        return list(set(itertools.chain.from_iterable([t._satisfied_rules() for t in self.titrators])))
    
    def can_titrate(self):
        return all([t.can_titrate() for t in self.titrators])
    
    def get_suggestion_texts(self):
        suggestions = [t.get_suggestion_texts() for t in self.titrators]
        return itertools.chain(suggestions)

## Some examples

### Testing dosing ladder

In [10]:
#|export

metoprolol_succinate = Ingredient("metoprolol succinate")
metoprolol_ladder = DosingLadder([
    Medication(metoprolol_succinate, "12.5 mg", "PO", "daily"),
    Medication(metoprolol_succinate, "25 mg", "PO", "daily"),
    Medication(metoprolol_succinate, "50 mg", "PO", "daily"),
    Medication(metoprolol_succinate, "100 mg", "PO", "daily"),
])

In [11]:
#|export

carvedilol = Ingredient("carvedilol")
carvedilol_ladder = DosingLadder([
    Medication(carvedilol, "3.125 mg", "PO", "BID"),
    Medication(carvedilol, "6.25 mg", "PO", "BID"),
    Medication(carvedilol, "12.5 mg", "PO", "BID"),
    Medication(carvedilol, "25 mg", "PO", "BID"),
])

In [12]:
#|export

bisoprolol = Ingredient("bisoprolol")
bisoprolol_ladder = DosingLadder([
    Medication(bisoprolol, "1.25 mg", "PO", "BID"),
    Medication(bisoprolol, "2.5 mg", "PO", "BID"),
    Medication(bisoprolol, "5 mg", "PO", "BID"),
    Medication(bisoprolol, "10 mg", "PO", "BID"),
])

In [13]:
metoprolol_ladder._at_highest_dose(Medication(metoprolol_succinate, "100 mg", "PO", "daily"))

True

In [14]:
metoprolol_ladder._at_highest_dose(Medication(metoprolol_succinate, "50 mg", "PO", "daily"))

False

In [15]:
metoprolol_ladder._current_dose_index(Medication(metoprolol_succinate, "50 mg", "PO", "daily"))

2

In [16]:
metoprolol_ladder.next_dose_up(Medication(metoprolol_succinate, "50 mg", "PO", "daily"))

metoprolol succinate 100 mg PO daily

In [17]:
metoprolol_ladder.next_dose_down(Medication(metoprolol_succinate, "50 mg", "PO", "daily"))

metoprolol succinate 25 mg PO daily

### Testing Rules

In [18]:
#|export

mild_hyperkalemia = NonTitrationLimitingIntolerance(
    'K', 'in', [n/10 for n in range(51, 56)],
    additional_success_actions=[Stop],
    )

In [19]:
p = Patient(K=5.3)
mild_hyperkalemia.meets_threshold(p)

True

In [20]:
mild_hyperkalemia.suggested_actions(p)

[__main__.Stop]

In [21]:
#|export

hypotension = TitrationLimitingIntolerance('SBP', 'lt', 90)
bradycardia = TitrationLimitingIntolerance('HR' , 'lt', 60)  # make it conditional
decompensation = TitrationLimitingIntolerance('decompensated', 'eq', True)
symptoms = TitrationLimitingIntolerance('symptomatic', 'eq', True)

### Testing titrators

In [22]:
#|export

class BetaBlockerTitrator(Titrator):
    intolerances = [
        hypotension,
        bradycardia,
        decompensation,
        symptoms,
    ]

class MetoprololTitrator(BetaBlockerTitrator):
    dosing_ladder = metoprolol_ladder

class CarvedilolTitrator(BetaBlockerTitrator):
    dosing_ladder = carvedilol_ladder

class BisoprololTitrator(BetaBlockerTitrator):
    dosing_ladder = bisoprolol_ladder

In [23]:
p = Patient(SBP=100, HR=80, decompensated=True, medications=[Medication(metoprolol_succinate, "25 mg", "PO", "daily")])
t = BetaBlockerTitrator(p, dosing_ladder=metoprolol_ladder)

In [24]:
t.can_titrate()

False

In [25]:
t._satisfied_rules()

[decompensated eq True]

In [26]:
t.get_suggested_actions()

In [27]:
t.get_suggestion_texts()

['Continue metoprolol succinate 25 mg PO daily.',
 'Decrease metoprolol succinate to metoprolol succinate 12.5 mg PO daily.',
 'Mark metoprolol succinate 25 mg PO daily as maximum tolerated dose.']

In [28]:
it_cond1 = ConditionalIntolerance(parameter='HR', operation='lt', threshold=60,
                                  condition_parameter='has_pacemaker',
                                  condition_operation='eq', condition_threshold=False)

In [29]:
d = dict()
d[Ingredient("lisinopril").name] = '1'

In [30]:
d

{'lisinopril': '1'}

In [56]:
bb_tg = TitratorGroup.from_titrator_types(p, [MetoprololTitrator, CarvedilolTitrator, BisoprololTitrator], exclusive=True)

In [57]:
bb_tg._satisfied_rules()

[decompensated eq True]

In [58]:
bb_tg.can_titrate()

False

In [64]:
bb_tg.get_suggestion_texts()

AttributeError: 'NoneType' object has no attribute 'dose'

In [45]:
beta_blocker_dosing_ladder = DosingLadderGroup([metoprolol_ladder, carvedilol_ladder, bisoprolol_ladder])

In [46]:
beta_blocker_dosing_ladder.lowest_doses

{'metoprolol succinate': metoprolol succinate 12.5 mg PO daily,
 'carvedilol': carvedilol 3.125 mg PO BID,
 'bisoprolol': bisoprolol 1.25 mg PO BID}

In [47]:
beta_blocker_dosing_ladder.highest_doses

{'metoprolol succinate': metoprolol succinate 100 mg PO daily,
 'carvedilol': carvedilol 25 mg PO BID,
 'bisoprolol': bisoprolol 10 mg PO BID}

In [1]:
from nbdev.export import nb_export

nb_export('titrations.ipynb')